In [46]:
from keras.models import model_from_json
import numpy as np
from keras.preprocessing import image
from keras.applications import vgg19
from pathlib import Path
import joblib

In [19]:
# Load keras VGG19 model that was pre-trained on against the ImageNet databse
model = vgg19.VGG19()

In [20]:
# Load the image file
# Resize it to 224*224 pixels (required by this model)
img = image.image_utils.load_img("./images/bay.jpg", target_size=(224, 224))

# Convert the image to numpy array
x = image.image_utils.img_to_array(img)

# Add fourth dimension since keras expects a list of images
x = np.expand_dims(x, axis=0)

In [21]:
# Normalize the input image's pixel values to the range used when training the neural network
x = vgg19.preprocess_input(x)

In [22]:
# Run the image through the deep neural network to perform prediction
predictions = model.predict(x)

1/1 [==============================] - 0s 296ms/step


In [23]:
# Look up the the names of the prediction classes.
# Index Zero is the result for the first image
predicted_classes = vgg19.decode_predictions(predictions)

In [24]:
# Print the top predictions for the image
for imagenet_id, name, likelihood in predicted_classes[0]:
    print(f"Prediction: {name} – {likelihood:.2f}")

Prediction: seashore – 0.60
Prediction: promontory – 0.23
Prediction: sandbar – 0.07
Prediction: lakeside – 0.04
Prediction: breakwater – 0.02


In [30]:
# Import training data
dog_path = Path("./images/training_data/dogs")
not_dog_path = Path("./images/training_data/not_dogs")

In [31]:
# Load images in to the images array and create labels (1 dog, 0 not dog)
images = []
labels = []

# Load all the dog images
for img in dog_path.glob("*.png"):
    # Load images from the disk
    img = image.image_utils.load_img(img)

    # Convert the image to numpy array
    image_array = image.image_utils.img_to_array(img)

    # Add the image to a list of images
    images.append(image_array)

    # For each dog image, the label should 1
    labels.append(1)

# Load all the not-dog images
for img in not_dog_path.glob("*.png"):
    # Load images from the disk
    img = image.image_utils.load_img(img)

    # Convert the image to numpy array
    image_array = image.image_utils.img_to_array(img)

    # Add the image to a list of images
    images.append(image_array)

    # For each not-dog image, the label should 0
    labels.append(0)

In [32]:
# Create a single numpy array with all the images loaded
X_train = np.array(images)

# Create a single numpy array with all the labels created
Y_train = np.array(labels)

# Normalize image data to (0 - 1) range
X_train = vgg19.preprocess_input(X_train)

# Load a pre-trained neural network to use as feature extractor
pre_trained_nn = vgg19.VGG19(weights="imagenet", include_top=False, input_shape=(64, 64, 3))

# Extract features for each image (all in one pass)
features_X = pre_trained_nn.predict(X_train)

2/2 [==============================] - 1s 294ms/step


In [33]:
# Save the the array of extracted features to a file
joblib.dump(features_X, "X_train.dat")

['X_train.dat']

In [34]:
# Save the matching array of expected values to a file
joblib.dump(Y_train, "Y_train.dat")

['Y_train.dat']

In [35]:
# Load the extracted features and matching expected values from the .dat files
X_train = joblib.load("X_train.dat")
Y_train = joblib.load("Y_train.dat")

In [36]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [37]:
# Create a model and add layers
model = Sequential()

# Flatten the convolutional layer input
model.add(Flatten(input_shape=X_train.shape[1:]))

# Add the dense layers
model.add(Dense(256, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation="sigmoid"))

In [38]:
# Compile the model
model.compile(
    optimizer="adam",
    metrics=["accuracy"],
    loss="binary_crossentropy"
)

In [40]:
# Train the model
model.fit(
    X_train,
    Y_train,
    epochs=10,
    shuffle=True
)

Epoch 1/10
2/2 [==============================] - 0s 3ms/step - loss: 1.1409e-22 - accuracy: 1.0000
Epoch 2/10
2/2 [==============================] - 0s 3ms/step - loss: 2.3233e-08 - accuracy: 1.0000
Epoch 3/10
2/2 [==============================] - 0s 4ms/step - loss: 4.5611e-21 - accuracy: 1.0000
Epoch 4/10
2/2 [==============================] - 0s 3ms/step - loss: 1.2676e-17 - accuracy: 1.0000
Epoch 5/10
2/2 [==============================] - 0s 3ms/step - loss: 4.7443e-17 - accuracy: 1.0000
Epoch 6/10
2/2 [==============================] - 0s 4ms/step - loss: 1.2627e-17 - accuracy: 1.0000
Epoch 7/10
2/2 [==============================] - 0s 2ms/step - loss: 4.1215e-11 - accuracy: 1.0000
Epoch 8/10
2/2 [==============================] - 0s 2ms/step - loss: 6.4255e-21 - accuracy: 1.0000
Epoch 9/10
2/2 [==============================] - 0s 2ms/step - loss: 2.6176e-19 - accuracy: 1.0000
Epoch 10/10
2/2 [==============================] - 0s 3ms/step - loss: 7.2730e-20 - accuracy: 1.0000

In [42]:
# Save the neural network structure
model_structure = model.to_json()
f = Path("./transfer-learning/model_structure.json")
f.write_text(model_structure)

1450

In [43]:
# Save the neural network trained weights
model.save_weights("./transfer-learning/model_weights.h5")

In [44]:
# Load the json file the contains the model structure
f = Path("./transfer-learning/model_structure.json")
model_structure = f.read_text()

In [47]:
# Create the keras model object from the model_structure json data
model = model_from_json(model_structure)

In [48]:
# Re-load the models trained weigths
model.load_weights("./transfer-learning/model_weights.h5")

In [65]:
# Load an image file to predict
# Resie the image to (64, 64) pixel image required by the model
img = image.image_utils.load_img("./images/not_dog.png", target_size=(64, 64))

# Convert the image to a numpy array
img_array = image.image_utils.img_to_array(img)

# Add a fourth dimesion to the image
# keras expects a list of images not a single image
images = np.expand_dims(img_array, axis=0)

In [66]:
# Normalize the data
images = vgg19.preprocess_input(images)

In [67]:
# use the pre-trained neural network to extract features from out test image
feature_extraction_model = vgg19.VGG19(weights="imagenet", include_top=False, input_shape=(64, 64, 3))

In [68]:
# Make predictions of features
features = feature_extraction_model.predict(images)

1/1 [==============================] - 0s 76ms/step


In [69]:
# Given the extracted features
# Make final predictions using the saved model
results = model.predict(features)

1/1 [==============================] - 0s 15ms/step


In [70]:
# Since we are only predicting one image with possible class
# We only need check the first result's first element
single_result = results[0][0]

In [71]:
# Print the results
print(f"Likelihood that this image contains a dog: {int(single_result * 100)}")

Likelihood that this image contains a dog: 0
